In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Предобработка

### Чтение всех данных (без знаков препинания)

In [ ]:
!pip install transformers
!pip install python-Levenshtein
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import torch
import Levenshtein
from transformers import AdamW, AutoTokenizer, T5TokenizerFast
from torch.optim.lr_scheduler import StepLR
import numpy as np


In [ ]:
import pandas as pd
dataset_ruSpell = pd.read_csv("/content/drive/MyDrive/HackMurmansk/RuSpellGoldDataset.csv")
# dataset_ruSpell = pd.read_csv("/content/drive/MyDrive/HackMurmansk/target_dataset_without_html.csv")
dataset_ruSpell

,Unnamed: 0,Неправильный вариант,Правильный вариант
0,0,"* Причины, на которых герцог Бассано основывал...","* Причины, на которых герцог Бассано основывал..."
1,1,"1812 г. А. Б. Куракин (1752–1818), будучи русс...","1812 г. А. Б. Куракин (1752–1818), будучи русс..."
2,2,"«А па-азвольте узнать, – заговорил он надменны...","«А па-азвольте узнать, – заговорил он надменны..."
3,3,А между тем для художника слова эти не имеют н...,А между тем для художника слова эти не имеют н...
4,4,"Без сомнения, он понимал, что, отрицая нравств...","Без сомнения, он понимал, что, отрицая нравств..."
...,...,...,...
3417,3417,Дмитрий Медведев хочет остаться на президентск...,Дмитрий Медведев хочет остаться на президентск...
3418,3418,"Обнаженные русские натурщицы Андре Кертеша, во...","Обнажённые русские натурщицы Андре Кертеша, во..."
3419,3419,ФСФР признала законным байбэк шахты «Распадска...,ФСФР признала законным байбэк шахты «Распадска...
3420,3420,Мэнни Пакьяо в преддверии боя с мексиканским б...,Мэнни Пакьяо в преддверии боя с мексиканским б...


### Проверка и чистка NANов

In [ ]:
dataset_ruSpell.isna().sum()

Unnamed: 0              0
Неправильный вариант    0
Правильный вариант      0
dtype: int64

In [ ]:
dataset_ruSpell = dataset_ruSpell.dropna()
dataset_ruSpell.isna().sum()

Unnamed: 0              0
Неправильный вариант    0
Правильный вариант      0
dtype: int64

### Проверка подключения GPU

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())

    print("We will use the GPU: ", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU:  Tesla T4


### Загрузка токенизатора и Токенизация данных

In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained('UrukHan/t5-russian-spell')

# Convert Pandas Series to lists of strings
sentences_with_errors = list(dataset_ruSpell["Неправильный вариант"])
corrected_sentences = list(dataset_ruSpell["Правильный вариант"])
dataset = dataset_ruSpell[["Неправильный вариант", "Правильный вариант"]].to_numpy()

In [ ]:

cnt_lev = 0
sum_len = 0
for idx, row in dataset_ruSpell.iterrows():
    cnt_lev += Levenshtein.distance(row['Неправильный вариант'], row['Правильный вариант'])
    sum_len += len(row["Неправильный вариант"])
cnt_lev / sum_len, cnt_lev, sum_len

(0.008945390840866738, 5290, 591366)

In [ ]:
from collections import Counter
import re

In [ ]:
non_letters = []
for sent in corrected_sentences:
  sent = re.sub(r"[а-яА-Я ёЁa-zA-Z\w\n]", "", sent)
  non_letters += list(sent)


In [ ]:
cnt = Counter(non_letters)
sorted(cnt.items(), key=lambda x: x[1], reverse=True)
allowed_punc = set([x[0] for x in cnt.items() if x[1] > 10])

In [ ]:
cnt

Counter({'*': 44,
         ',': 7840,
         '.': 4800,
         '(': 360,
         '–': 332,
         ')': 424,
         '«': 950,
         '-': 1572,
         '?': 250,
         '»': 952,
         ';': 312,
         '!': 242,
         ':': 514,
         '…': 106,
         '—': 224,
         '„': 4,
         '“': 30,
         '"': 176,
         '[': 6,
         ']': 6,
         '”': 22,
         '=': 8,
         '®': 2,
         '%': 138,
         '+': 24,
         '/': 90,
         '$': 40,
         '•': 10,
         '✔': 8,
         '@': 2,
         '№': 16,
         '&': 8,
         '’': 2,
         '̆': 2,
         '°': 6,
         "'": 12,
         '£': 2,
         '‘': 2,
         '₽': 2,
         '~': 2,
         '^': 4})

#### Зададим всем предложениям фиксированную длину с использованием тега [SEP], демонстрирующий конец предложения, а далее с помощью токена [PAD] добавляем нули, чтобы предложения приобрели одинаковую длину

Посчитаем максимальные длины предложений в обоих списках

In [ ]:
for data, name in zip([sentences_with_errors, corrected_sentences], ['Incorrect', 'Correct']):
  max_len = 0
  for sent in data:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
  print(name, ":", max_len)

Incorrect : 171
Correct : 173


Приведем все предложения к общей длине, для неправильных предложений получим маски внимания и преобразуем все в тензоры

### Архитектура модели

In [ ]:
class CustomModel(torch.nn.Module):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.t5model = model

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.t5model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True
        )

        return outputs

### Токенизация и разбиение выборки

In [ ]:
task_prefix = "Spell correct: "
tokenizer = T5TokenizerFast.from_pretrained('UrukHan/t5-russian-spell')
class CustomDataset(torch.utils.data.Dataset):

  def __init__(self, inputs=None, answers=None, tokenizer=None, task_prefix="Spell correct: "):
    super().__init__()
    self.input = inputs
    self.answers = answers
    self.tokenizer = tokenizer
    self.task_prefix = task_prefix

  def __getitem__(self, idx):
    input_s = self.input[idx]
    true_s = self.answers[idx]

    # input_encoded = self._transform(input_s, task_prefix=True)
    # input_encoded['input_ids'][input_encoded['input_ids'] == tokenizer.pad_token_id] = 0
    source_encoding = self.tokenizer(
            task_prefix + input_s,
            max_length=256,
            padding= "max_length",
            truncation= "only_second",
            return_attention_mask=True,
            # add_special_tokens=True,
            return_tensors= "pt"
        )

    input_encoded = source_encoding["input_ids"]
    attention_mask = source_encoding["attention_mask"]

    target_encoding = self.tokenizer(
            true_s,
            max_length=256,
            padding= "max_length",
            truncation= True,
            return_attention_mask=True,
            # add_special_tokens=True,
            return_tensors= "pt"
        )
    true_encoded = target_encoding["input_ids"]
    true_encoded[true_encoded==0] = -100

    sample = {
        'input': input_s,
        'true_label': true_s,
        'input_ids': input_encoded,
        'labels': true_encoded,
        'attention_mask': attention_mask
    }
    return sample

  def __len__(self):
    return len(self.input)


In [ ]:
#create datasets

train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset_in_out, val_dataset_in_out = random_split(dataset, [train_size, val_size])
train_dataset_in_out = np.array(train_dataset_in_out)
val_dataset_in_out = np.array(val_dataset_in_out)

train_dataset = CustomDataset(inputs=list(train_dataset_in_out[:, 0]), answers=list(train_dataset_in_out[:, 1]), tokenizer=tokenizer)
valid_dataset = CustomDataset(inputs=list(val_dataset_in_out[:, 0]), answers=list(val_dataset_in_out[:, 1]), tokenizer=tokenizer)


In [ ]:
train_dataset_in_out.shape

(2566, 2)

###  Обучение

In [ ]:
 # Define training parameters
epochs = 1
batch_size = 8

# Create data loaders for training and validation
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    # shuffle= True,
    num_workers=4,
    batch_size=batch_size,
)

validation_dataloader = DataLoader(
    valid_dataset,
    # shuffle= True,
    num_workers=4,
    sampler=RandomSampler(valid_dataset), #SequentialSampler
    batch_size=batch_size
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Initialize the model
import torch.optim as optim
from transformers.optimization import Adafactor, AdafactorSchedule
from transformers import AdamW, T5ForConditionalGeneration

# custom_model = CustomModel(AutoModelForSeq2SeqLM.from_pretrained('UrukHan/t5-russian-spell'))
custom_model = AutoModelForSeq2SeqLM.from_pretrained('UrukHan/t5-russian-spell')

# modules_to_freeze = [custom_model.t5model.encoder, custom_model.t5model.decoder, custom_model.t5model.lm_head]
# modules_to_freeze = [custom_model.encoder, custom_model.decoder, custom_model.lm_head]

# Freeze pre-trained encoder layers
for param in custom_model.encoder.parameters():
    param.requires_grad = False

# Fine-tune decoder layers
for param in custom_model.decoder.parameters():
    param.requires_grad = False#True

# Fine-tune the language modeling head
for param in custom_model.lm_head.parameters():
    param.requires_grad = True

custom_model = custom_model.to(device)
# custom_model.resize_token_embeddings(len(tokenizer))
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=0.0001)


In [ ]:
from collections import Counter
import re
def trim_string_to_ratio(original_text, modified_text, max_ratio=1.1):
    modified_text = re.sub(r'</s>.*', '', modified_text)
    original_length = len(original_text)
    cleaned_text = modified_text[: int(max_ratio * original_length)]
    cleaned_text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9 \w]{2,}', '', cleaned_text)
    cleaned_text = re.sub(r' {2,}', ' ', cleaned_text)

    non_letters = []
    for sent in corrected_sentences:
      sent = re.sub(r"[а-яА-Я ёЁa-zA-Z\w\n]", "", sent)
      non_letters += list(sent)

    cnt = Counter(non_letters)
    sorted(cnt.items(), key=lambda x: x[1], reverse=True)
    allowed_punc = set([x[0] for x in cnt.items() if x[1] > 10])

    cleaned_text = ''.join(char for char in cleaned_text if char.isalpha() or char.isalnum() or char in allowed_punc or char.isspace())
    return cleaned_text

In [ ]:
torch.cuda.empty_cache()
flag = True
for epoch in range(epochs):
    flag = True
    custom_model.train()
    total_loss = 0.0
    total_train_levenshtein_distance = 0
    total_train_reference_length = 0

    # Initialize progress bar
    progress_bar = tqdm(enumerate(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs} - Training', total=len(train_dataloader))

    for step, batch in progress_bar:
        inputs, true_labels, input_ids, labels, attention_mask = batch['input'], batch['true_label'], batch['input_ids'], batch['labels'], batch['attention_mask']
        input_ids = torch.reshape(input_ids, (batch_size, -1))
        labels = torch.reshape(labels, (batch_size, -1))
        attention_mask = torch.reshape(attention_mask, (batch_size, -1))
        input_ids, labels, attention_mask = input_ids.to(device), labels.to(device), attention_mask.to(device)

        optimizer.zero_grad()
        result_model = custom_model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)

        logits, loss = result_model.logits, result_model.loss

        optimizer.step()
        total_loss += loss.detach().item()

        # Convert logits to predicted token IDs
        predicted_token_ids = torch.argmax(logits, dim=-1)

        # Convert token IDs to text
        decode_logits = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=False, clean_up_tokenization_spaces=True)

        for gen_sent, ref_sent in zip(decode_logits, true_labels):
            gen_sent = trim_string_to_ratio(ref_sent, gen_sent)
            total_train_levenshtein_distance += Levenshtein.distance(gen_sent, ref_sent)
            total_train_reference_length += len(ref_sent)

        # Update progress bar description with the current loss and Levenshtein distance
        progress_bar.set_description(f'Epoch {epoch + 1}/{epochs} - Training - Loss: {loss.item():.4f} - Levenshtein: {total_train_levenshtein_distance / total_train_reference_length:.4f}')

    # Calculate the average Levenshtein distance per character on the training dataset
    average_train_loss = total_loss / len(train_dataloader)
    average_train_levenshtein_distance = total_train_levenshtein_distance / total_train_reference_length

    print(f"Epoch [{epoch + 1}/{epochs}] - Train Loss: {average_train_loss:.4f} - Train Levenshtein Distance: {average_train_levenshtein_distance:.4f}")

    # Validation
    custom_model.eval()
    total_val_levenshtein_distance = 0
    total_val_reference_length = 0
    total_val_loss = 0

    with torch.no_grad():
      for batch in tqdm(validation_dataloader, desc=f'Epoch {epoch + 1}/{epochs} - Validation', total=len(validation_dataloader)):
          inputs, true_labels, input_ids, labels, attention_mask = batch['input'], batch['true_label'], batch['input_ids'], batch['labels'], batch['attention_mask']
          input_ids = torch.reshape(input_ids, (batch_size, -1))
          labels = torch.reshape(labels, (batch_size, -1))
          attention_mask = torch.reshape(attention_mask, (batch_size, -1))
          input_ids, labels, attention_mask = input_ids.to(device), labels.to(device), attention_mask.to(device)

          result_model = custom_model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)

          logits, val_loss = result_model.logits, result_model.loss

          total_val_loss += val_loss.detach().item()

          # Convert logits to predicted token IDs
          predicted_token_ids = torch.argmax(logits, dim=-1)

          # Convert token IDs to text
          decode_logits = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=False, clean_up_tokenization_spaces=True)

          for gen_sent, ref_sent in zip(decode_logits, true_labels):
              gen_sent = trim_string_to_ratio(ref_sent, gen_sent)
              total_val_levenshtein_distance += Levenshtein.distance(gen_sent, ref_sent)
              total_val_reference_length += len(ref_sent)
              if flag:
                  print("\n", gen_sent, "\n", ref_sent, "\n", total_train_levenshtein_distance)
          flag = False

    # Calculate the average Levenshtein distance per character on the validation dataset
    average_val_loss = total_val_loss / len(validation_dataloader)
    average_val_levenshtein_distance = total_val_levenshtein_distance / total_val_reference_length

    print(f"Epoch [{epoch + 1}/{epochs}] - Validation Loss: {average_val_loss:.4f} - Validation Levenshtein Distance: {average_val_levenshtein_distance:.4f}")

    # Adjust the learning rate at the beginning of each epoch

    torch.cuda.empty_cache()

# Save the trained model
torch.save(custom_model.state_dict(), "spelling_and_punctuation_correction_model.pth")


Epoch 1/1 - Training - Loss: 2.0560 - Levenshtein: 0.0706: 100%|██████████| 321/321 [06:23<00:00,  1.19s/it]


Epoch [1/1] - Train Loss: 0.4099 - Train Levenshtein Distance: 0.0706


Epoch 1/1 - Validation:   0%|          | 0/107 [00:00<?, ?it/s]


 В СПб отлично с постстанов-ией. еслищас бы в 2020 в в виде декорации на Хэллоуин ставить огромную летучую мышь в центре города 
 В СПб отлично с постиронией, щас бы в 2020м в виде декорации на Хэллоуин ставить огромную летучую мышь в центре города  
 30823

 В США вышла в прокат новая версия фильма о Дэдпуле под названием "Жил-был Дэдпул которая провалилась в американском прокате. На отцензурированную ленту обрушился шквал критики со стороны поклонников кинокомикса. 
 В США вышла в прокат новая версия фильма о Дэдпуле под названием «Жил-был Дэдпул», которая провалилась в американском прокате. На отцензурированную ленту обрушился шквал критики со стороны поклонников кинокомикса. 
 30823

 Согласно отчету, опубликованному аналитической компанией в области мобильных приложений Fluurry, которая принадлежит ahahoo, показатели использования мобильных приложений в 2014 году выросли на 76% по сравнению с 2013 годом. Интересно, что наибольший рост, который составил более 100% за год, показали

Epoch 1/1 - Validation:   1%|          | 1/107 [00:01<02:05,  1.18s/it]


 Данные звуковые электросирены оповещают 12,3 тыс. человек в г. Южа, 90% населения города. 
 Данные звуковые электросирены оповещают 12,3 тыс. человек в г. Южа или 90% населения города. 
 30823

 Так, например, доктор Карл Бартель, представитель одной из ведущих немецких девелоперских компаний в сегменте коммерческой недвижимости Ebertz Part Partner, отметил, что они в ExpoREAL принимают участие уже пятый год подряд и очень довольны результатами 
 Так, например, доктор Карл Бартель, представитель одной из ведущих немецких девелоперских компаний в сегменте коммерческой недвижимости Ebertz & Partner, отметил, что они в ExpoREAL принимают участие уже пятый год подряд и очень довольны результатами. 
 30823


Epoch 1/1 - Validation: 100%|██████████| 107/107 [02:04<00:00,  1.17s/it]


Epoch [1/1] - Validation Loss: 0.3388 - Validation Levenshtein Distance: 0.0485


### Запуск предобученной модели

In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
import pandas as pd
import torch

def test(dataset):
    MODEL_NAME = 'UrukHan/t5-russian-spell'
    loaded_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

    # Load the saved weights
    model_path = "spelling_and_punctuation_correction_model.pth"
    loaded_model.load_state_dict(torch.load(model_path))
    loaded_model.eval()

    # Load the tokenizer
    tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)

    # Your dataset and input sequences
    dataset = dataset.dropna()
    input_sequences = dataset["Неправильный вариант"]

    task_prefix = "Spell correct: "

    # Tokenize and predict
    MAX_INPUT = 256
    encoded = tokenizer(
        [task_prefix + sequence for sequence in input_sequences],
        padding="max_length",
        max_length=MAX_INPUT,
        truncation=True,
        return_tensors="pt",
    )

    with torch.no_grad():
        predicts = loaded_model.generate(encoded['input_ids'])  # Generate predictions

    decode_prediction = tokenizer.batch_decode(predicts, skip_special_tokens=True)  # Decode predictions

    return decode_prediction

dataset = pd.read_csv("/content/drive/MyDrive/HackMurmansk/RuSpellGoldDataset.csv")
predictions = test(dataset[:20])

In [ ]:
for res in predictions:
  print(res, "\n")

* Причины, на которых герцог Бассано основывал свой отказ выдать свои паспорта, никогда не могли бы заставить меня предполагать, чтобы поступок моего посла послужил поводом к нападению. 

1812 г. А. Б. Куракин (1752–1818), будучи русским послом в Париже, после неудачных переговоров о выводе французских войск из Пруссии 29 апреля (11 мая) 1812 г. потребовал выдать паспорта ему и всему персоналу посольства для возвращения в Россию. 

«А позвольте узнать, – заговорил он надменным голосом. – По какому праву вы здесь, а-а, хотите, мой друг??» Незнакомец говорил необыкновенно быстро, отрывочно и в нос. 

А между тем для художника слова эти не имеют никакого смысла и даже не затрагивают самого события. 

Без сомнения, он понимал, что, отрицая нравственную ответственность личности, он тем самым отрицает и свободу ее. Но он верил всем существом своим (гораздо слепее Герцена, который, кажется, под конец усомнился), что социализм не только не разрушает свободу личности, а, напротив, восстанавлива